# Retail Data Analysis

This notebook performs a comprehensive analysis of retail data, including SQL queries, EDA, visualizations, and business insights.

## A. Basic SQL Queries

1. Retrieve all stock items that contain the word "T-LIGHT" in their description.
2. Calculate the total quantity sold per StockCode.
3. Find total revenue (Quantity × UnitPrice) per CustomerID.
4. Get a list of all invoices and count of distinct stock items per invoice.
5. Perform an inner join between sales and stock details to display full item names along with total revenue per item.

## B. EDA Using Pandas

6. Check for missing values, data types, and duplicates in both datasets.
7. Convert InvoiceDate into datetime, and extract: Invoice date, Month, Hour of transaction.
8. Add a new column TotalPrice = Quantity × UnitPrice.
9. Identify the top 3 bestselling items by quantity sold.
10. Find out how many unique customers made purchases and the average quantity per invoice.

## C. Visualization

11. Plot a bar chart of top 10 items by quantity sold.
12. Plot total sales per hour of the day to understand peak shopping hours.
13. Create a pie chart showing revenue distribution among top 5 customers.
14. Visualize the monthly revenue trend using a line chart.
15. Create a stacked bar chart of top 5 invoices by revenue, showing contribution from each item.

## D. Business Insights

16. Identify which product generates the highest total revenue.
17. Determine average order value per invoice.
18. Find the customer with the highest number of transactions.
19. Evaluate the percentage of stock items sold vs. total in inventory.
20. Check how many products in the sales data do not have a matching entry in the stockDetails file.


In [ ]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

# Load the CSV files into DataFrames
sales_df = pd.read_csv('sales 6.csv')
stock_df = pd.read_csv('StockDetails 6.csv')

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('retail_data.db')

# Load DataFrames into SQLite database
sales_df.to_sql('sales', conn, if_exists='replace', index=False)
stock_df.to_sql('stock', conn, if_exists='replace', index=False)

# A. Basic SQL Queries

# 1. Retrieve all stock items that contain the word "T-LIGHT" in their description.
query1 = """
SELECT * FROM stock
WHERE Description LIKE '%T-LIGHT%'
"""
result1 = pd.read_sql(query1, conn)
print(result1)

# 2. Calculate the total quantity sold per StockCode.
query2 = """
SELECT StockCode, SUM(Quantity) as TotalQuantity
FROM sales
GROUP BY StockCode
"""
result2 = pd.read_sql(query2, conn)
print(result2)

# 3. Find total revenue (Quantity × UnitPrice) per CustomerID.
query3 = """
SELECT CustomerID, SUM(Quantity * UnitPrice) as TotalRevenue
FROM sales
GROUP BY CustomerID
"""
result3 = pd.read_sql(query3, conn)
print(result3)

# 4. Get a list of all invoices and count of distinct stock items per invoice.
query4 = """
SELECT InvoiceNo, COUNT(DISTINCT StockCode) as DistinctStockItems
FROM sales
GROUP BY InvoiceNo
"""
result4 = pd.read_sql(query4, conn)
print(result4)

# 5. Perform an inner join between sales and stock details to display full item names along with total revenue per item.
query5 = """
SELECT s.StockCode, st.Description, SUM(s.Quantity * s.UnitPrice) as TotalRevenue
FROM sales s
INNER JOIN stock st ON s.StockCode = st.StockCode
GROUP BY s.StockCode, st.Description
"""
result5 = pd.read_sql(query5, conn)
print(result5)

# B. EDA Using Pandas

# 6. Check for missing values, data types, and duplicates in both datasets.
missing_values_sales = sales_df.isnull().sum()
missing_values_stock = stock_df.isnull().sum()
data_types_sales = sales_df.dtypes
data_types_stock = stock_df.dtypes
duplicates_sales = sales_df.duplicated().sum()
duplicates_stock = stock_df.duplicated().sum()
print(missing_values_sales, missing_values_stock, data_types_sales, data_types_stock, duplicates_sales, duplicates_stock)

# 7. Convert InvoiceDate into datetime, and extract: Invoice date, Month, Hour of transaction
sales_df['InvoiceDate'] = pd.to_datetime(sales_df['InvoiceDate'], dayfirst=True)
sales_df['InvoiceDateOnly'] = sales_df['InvoiceDate'].dt.date
sales_df['InvoiceMonth'] = sales_df['InvoiceDate'].dt.month
sales_df['InvoiceHour'] = sales_df['InvoiceDate'].dt.hour

# 8. Add a new column TotalPrice = Quantity × UnitPrice.
sales_df['TotalPrice'] = sales_df['Quantity'] * sales_df['UnitPrice']

# 9. Identify the top 3 bestselling items by quantity sold.
top_3_bestselling_items = sales_df.groupby('StockCode')['Quantity'].sum().nlargest(3)
print(top_3_bestselling_items)

# 10. Find out how many unique customers made purchases and the average quantity per invoice.
unique_customers = sales_df['CustomerID'].nunique()
average_quantity_per_invoice = sales_df.groupby('InvoiceNo')['Quantity'].mean().mean()
print(unique_customers, average_quantity_per_invoice)

# C. Visualization

# 11. Plot a bar chart of top 10 items by quantity sold.
top_10_items = sales_df.groupby('StockCode')['Quantity'].sum().nlargest(10)
plt.figure(figsize=(10, 6))
top_10_items.plot(kind='bar')
plt.title('Top 10 Items by Quantity Sold')
plt.xlabel('StockCode')
plt.ylabel('Quantity Sold')
plt.show()

# 12. Plot total sales per hour of the day to understand peak shopping hours.
sales_per_hour = sales_df.groupby('InvoiceHour')['TotalPrice'].sum()
plt.figure(figsize=(10, 6))
sales_per_hour.plot(kind='bar')
plt.title('Total Sales per Hour of the Day')
plt.xlabel('Hour of the Day')
plt.ylabel('Total Sales')
plt.show()

# 13. Create a pie chart showing revenue distribution among top 5 customers.
top_5_customers = sales_df.groupby('CustomerID')['TotalPrice'].sum().nlargest(5)
plt.figure(figsize=(8, 8))
top_5_customers.plot(kind='pie', autopct='%1.1f%%')
plt.title('Revenue Distribution Among Top 5 Customers')
plt.ylabel('')
plt.show()

# 14. Visualize the monthly revenue trend using a line chart.
monthly_revenue = sales_df.groupby('InvoiceMonth')['TotalPrice'].sum()
plt.figure(figsize=(10, 6))
monthly_revenue.plot(kind='line')
plt.title('Monthly Revenue Trend')
plt.xlabel('Month')
plt.ylabel('Total Revenue')
plt.show()

# 15. Create a stacked bar chart of top 5 invoices by revenue, showing contribution from each item.
top_5_invoices = sales_df.groupby('InvoiceNo')['TotalPrice'].sum().nlargest(5).index
top_5_invoices_data = sales_df[sales_df['InvoiceNo'].isin(top_5_invoices)]
stacked_data = top_5_invoices_data.pivot_table(index='InvoiceNo', columns='StockCode', values='TotalPrice', aggfunc='sum').fillna(0)
stacked_data.plot(kind='bar', stacked=True, figsize=(10, 6))
plt.title('Top 5 Invoices by Revenue (Stacked by Item)')
plt.xlabel('InvoiceNo')
plt.ylabel('Total Revenue')
plt.show()

# D. Business Insights

# 16. Identify which product generates the highest total revenue.
highest_revenue_product = sales_df.groupby('StockCode')['TotalPrice'].sum().idxmax()
print(highest_revenue_product)

# 17. Determine average order value per invoice.
average_order_value = sales_df.groupby('InvoiceNo')['TotalPrice'].sum().mean()
print(average_order_value)

# 18. Find the customer with the highest number of transactions.
customer_highest_transactions = sales_df['CustomerID'].value_counts().idxmax()
print(customer_highest_transactions)

# 19. Evaluate the percentage of stock items sold vs. total in inventory.
total_stock_items = stock_df['StockCode'].nunique()
sold_stock_items = sales_df['StockCode'].nunique()
percentage_sold = (sold_stock_items / total_stock_items) * 100
print(percentage_sold)

# 20. Check how many products in the sales data do not have a matching entry in the stockDetails file.
unmatched_products = sales_df[~sales_df['StockCode'].isin(stock_df['StockCode'])]['StockCode'].nunique()
print(unmatched_products)
